In [ ]:
import random
import utils
from math import ceil
import time
import matplotlib.pyplot as plt


In [ ]:
primeArray = utils.read_data('primes.txt')

In [ ]:
def Encrypt(message, n, e):
    cipher = ""
    noChar = utils.getNumberOfChars(n)
    maxCipherLength = len(str(n-1)) 
    for i in range(0, len(message), noChar):
        blockToEncrypt = utils.ConvertToInt(message[:noChar]) #indexing outofbounds works fine in python
        cipherdBlock = str(utils.PowerMod(blockToEncrypt, e, n))
        if (len(cipherdBlock) < maxCipherLength):  #make sure all cipher blocks are the same length
            cipherdBlock = cipherdBlock.zfill(maxCipherLength)
        cipher += cipherdBlock
        message = message[noChar:]
    return cipher


In [ ]:
def Decrypt(cipher,n, d):
    maxCipherLength = len(str(n-1)) 
    message = ""
    for i in range(0, len(cipher), maxCipherLength):
        blockToDecrypt = int(cipher[:maxCipherLength])
        message += utils.ConvertToStr(utils.PowerMod(blockToDecrypt, d, n))
        cipher = cipher[maxCipherLength:]
    return message

In [ ]:

#implement RSA Algorithm to generate public and private key
def RSA_key_generator(p,q):
    n = p*q
    phi = (p-1)*(q-1)
    e = random.randint(2,phi-1)     #choose e randomly from 1 to phi
    while utils.GCD(e,phi) != 1:
        e = random.randint(2,phi-1)
    d = utils.InverseModulo(e,phi)
    return (e,d,n)

In [ ]:
#generate two random integers from the array of prime numbers
def generate_two_prime_numbers(prime_array):
    p = random.choice(prime_array)
    q = random.choice(prime_array)
    while p == q:
        q = random.choice(prime_array)
    return (p,q)

In [ ]:
#genertate two random integers from the array of prime numbers
(p,q) = generate_two_prime_numbers(primeArray)

#generate public and private key
(e,d,n) = RSA_key_generator(p,q)



message = "Hello World"



#encrypt the message
cypher = Encrypt(message,n,d)



#decrypt the message
decrypted_message = Decrypt(cypher,n,e)


#print the  decrypted message

print(decrypted_message)


In [ ]:
'''
TODO:
1)calculate encryption time for different key lenghts 
2)plot a graph of RSA encryption time vs. Key length

Note :in (1) IDK how big n should be (or how you should increase it every time) to statisfy the TA LOL
'''
# the code below doesn't work correctly but it might help 
# message = "a long message"
# e = []
# n = []
# for i in range(0, 99998):
#     #generate public and private key using each concecutive prime number in the prime array
#     (e_i,d,n_i) = RSA_key_generator(primeArray[i],primeArray[i+1])
#     e.append(e_i)
#     n.append(n_i)
# #encrypt the message using each pair of n and e and calculate the time it takes to encrypt store it in a list
# time_list = []   
# for i in range(0,len(e)):
#     start = time.time()
#     cypher = Encrypt(message,n[i],e[i])
#     end = time.time()
#     time_list.append(end-start)
# #plot time vs. n
# print(len(primeArray))
# print(max(time_list))
# plt.plot(n,time_list)




In [ ]:
'''
TODO:
1)Implement brute force (mathematical attack) on RSA algorithm using different values for n
2)plot a graph of Time to break the private key (in seconds) versus value of n. Discuss the results you obtain.

'''




In [ ]:
'''
TODO:
Implement the Chosen Cipher Text attack for RSA and justify why it happens in your report.
Note : you will find an explanation of the chosen cipher text attack in CS (RSA CCA.pdf) this file came 
with the project document IDk if it will be of any help. 
'''